In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv',encoding='latin1')


In [ ]:
print('TOTAL SAMPLE NUMBER :',len(data))

data[:5]

In [ ]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data[:5]

In [ ]:
data.info()

In [ ]:
data.isnull().values.any()

In [ ]:
data['v2'].nunique(), data['v1'].nunique()

In [ ]:
data['v1'].value_counts().plot(kind='bar');

In [ ]:
print("ToTal Sample Number:", len(data))

In [ ]:
data['v1'].value_counts().plot(kind='bar');

In [ ]:
print(data.groupby('v1').size().reset_index(name='count'))

In [ ]:
X_data = data['v2']
Y_data = data['v1']
print('Number of Mail {}'.format(len(X_data)))
print('Number of Label {}'.format(len(y_data)))

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data) # Line 5572 tokenizing
sequences = tokenizer.texts_to_sequences(X_data) # Convert to Index

In [ ]:
print(sequences[:5])

In [ ]:
word_to_index = tokenizer.word_index
print(word_to_index)

In [ ]:
word_to_counts = tokenizer.word_counts.items()
print(word_to_counts)

In [ ]:
threshold = 2
total_cnt = len(word_to_index) # Number of Word
rare_cnt = 0 # Frequency of Threshold for small word counts
total_freq = 0 # Training Data for total number of word for sum
rare_freq = 0 # Sum of number for small counts 

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # Frequency of Words less than threshold
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('Frequency of numbers %s번 Under rare value: %s'%(threshold - 1, rare_cnt))
print("Rate of rare words of vocabulary:", (rare_cnt / total_cnt)*100)
print("Percentage of rare words of overall frequency:", (rare_freq / total_freq)*100)

In [ ]:
vocab_size = len(word_to_index) + 1
print('Size of Words: {}'.format((vocab_size)))

In [ ]:
n_of_train = int(len(sequences) * 0.8)
n_of_test = int(len(sequences) - n_of_train)
print('number of training data:', n_of_train)
print('number of test data:', n_of_test)

In [ ]:
X_data = sequences
print('Maximum length of Mail : %d' % max(len(l) for l in X_data))
print('Average length of Mail : %f' % (sum(map(len, X_data))/len(X_data)))
plt.hist([len(s) for s in X_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
max_len = 189
# Length of Total Dataset is fit into max_len
data = pad_sequences(X_data, maxlen = max_len)
print("Size of Training Data(Shape)", data.shape)

In [ ]:
X_test = data[n_of_train:] # In X_Data, store just 1034 dataset
y_test = np.array(y_data[n_of_train:]) # In y_data, we have to add 1034 

X_train = data[:n_of_train] # In X_train, let's load 4135 dataset
y_train = np.array(y_data[:n_of_train])

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 32)) # vector size is 32
model.add(SimpleRNN(32)) # hidden size is 32
model.add(Dense(1, activation ='sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(X_train, y_train, epochs = 4, batch_size = 64, validation_split = 0.2)

In [ ]:
print("\n Test Accuracy %.4f" %(model.evaluate(X_test, y_test)[1]))

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()